## <font color=pink>피드백</font>

<font color=pink>
1. 우선 굳이 train, test 분리가 필요없어 보입니다.<br>
2. 제 사견으로는 영상의 조회수에 영향을 주는 요소가 너무 많고, 복잡하게 얽혀있기 때문에 이와 같은 방식은 적절한 결과를 도출해내기 어려울 것 같습니다.<br>
3. 제목만을 사용하고자 하신다면, 조회수가 높은 영상들의 문서유사도(제목간)등을 고려함으로써 해당 채널에서 조회수를 높이는 latent variable을 찾아내는 방식으로 해보심이 어떨까 싶습니다.<br>
4. 3에 대해 첨언하여 주제분석의 방향을 고려해보시면 좋을 것 같습니다. 하나의 채널에서 주 컨텐츠가 있지만, 여러가지 카테고리가 있습니다. 먹방이라도, ASMR이 있을 수 있고, 많이 먹는 영상이 있을 수 있죠 이들은 제목에 반영될 것이고, 조회수를 잘 뽑아내는 제목이 가지는 특성을 주제분석을 통해서 찾아낼 수 있지 않을까 싶습니다.<br>
5. 단어의 빈도만을 이용하는 방식은 한계가 많고, 명확합니다. nlp쪽 최근 동향과 발전추이를 공부하신다면 더 나은 분석이 가능해지실 것 같습니다.<br>
</font>

# 비타민 7기 12주차 복습과제
## 침착맨 Youtube 제목을 통한 조회수 예측
## 비타민 7기 이준석

 아무래도 Youtube에 크롤링할 흥미로운 데이터가 많을 것 같아 Youtube위주로 생각하다가 영상의 제목과 조회수의 관계에 대해 생각하게 되었습니다. 처음에는 Youtube의 인기 동영상 항목에 존재하는 영상들을 생각했지만, 최근 Youtube 영상들의 트렌드가 썸네일에 대부분의 정보들이 담겨있고 텍스트 타이틀에는 큰 의미가 없는 내용들 (예를 들어 요리 영상이지만 '이것만은 하지 말아주세요')을 담는 것이라 한계가 있다고 판단했습니다.  
 여기서 제가 즐겨 시청하는 채널인 '침착맨'님의 영상으로 범위를 좁혀보자는 생각이 들었습니다. '침착맨'님의 영상들은 타이틀의 텍스트로도 매우 정확한 정보들을 얻을 수 있기 떄문이었습니다. (예를 들어 '맛알못 침착맨의 초밥 먹방')  
 '침착맨' 영상의 타이틀을 크롤링하여 konlpy를 이용해 BOW를 생성한뒤, 이를 통해 조회수와 연관시킬수 있지 않을까 하고 생각했습니다.  
 미리 결론부터 말씀드리자면, 크롤링은 진행했지만 분석에 있어선 큰 진전을 보이지 못했습니다. 영상 제목과 조회수의 연관성을 알기 위해선 어떻게 해야할지 제 부족한 과제물 보시고 피드백부탁드립니다 ㅎㅎ,,

### 1. 데이터 크롤링

In [1]:
# Youtube는 BeautifulSoup으로 크롤링이 잘 진행되지 않아 Selenium을 통한 크롤링 진행
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()

In [2]:
# 침착맨 유튜브 채널 url
url = "https://www.youtube.com/user/zilioner83/videos"
driver.get(url.format(4))

침착맨 Youtube 채널에서 과거 영상 목록들을 로드하기 위해선 페이지 하단으로 스크롤이 필요하다.

In [3]:
import datetime

# whileSeconds초 동안 페이지를 스크롤하는 함수.
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, 200000);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [74]:
# 250초 정도 스크롤하면 4년 전 영상까지 로드 가능
doScrollDown(250)

In [8]:
# 침착맨 영상 제목, 길이, 조회수를 저장할 리스트
video_title = []
video_length = []
views = []

In [5]:
# 영상 제목, 조회수, 영상 길이를 담고 있는 태그들 선택
cssid_video_title = driver.find_elements_by_css_selector('#video-title')
cssid_text = driver.find_elements_by_css_selector('#text')

In [6]:
# 영상 길이를 담고있는, 즉 id='text'인 태그가 다른 태그에비해 갯수가 많은것을 확인할 수 있음.
print('cssid_text 리스트의 길이: ', len(cssid_text) )
print('cssid_video_title 리스트의 길이: ', len(cssid_video_title) )

cssid_text 리스트의 길이:  9367
cssid_video_title 리스트의 길이:  4680


In [7]:
# print를 통해 영상 길이가 어떠한 패턴으로 위치해있느지 파악
for i in range(20):
    print('idx :', i, ' text: ', cssid_text[i].text)


idx : 0  text:  탐색 건너뛰기
idx : 1  text:  로그인
idx : 2  text:  침착맨
idx : 3  text:  구독
idx : 4  text:  모두 재생
idx : 5  text:  13:43
idx : 6  text:  
idx : 7  text:  19:54
idx : 8  text:  
idx : 9  text:  21:44
idx : 10  text:  
idx : 11  text:  17:08
idx : 12  text:  
idx : 13  text:  32:05
idx : 14  text:  
idx : 15  text:  17:35
idx : 16  text:  
idx : 17  text:  1:04:27
idx : 18  text:  
idx : 19  text:  33:32


In [20]:
# 위 패턴을 이용해 조회수 저장
for i in range(5, len(cssid_text)-2, 2):
    video_length.append(cssid_text[i].text)

In [9]:
# 영상 제목과, 조회수를 저장
# 조회수는 속성값을 가져와 리스트화, 마지막 인덱스에 저장되어있는 것을 사용
for block in cssid_video_title:
    video_title.append(block.text)
    views.append(block.get_attribute("aria-label").split(" ")[-1])

### 02. 데이터 가공

In [10]:
# 예를 들어 '111,111회' 와 같은 문자열을 111111이라는 정수로 변환하여 저장
for idx, view in enumerate(views):
    tmp = view.replace("회", "")
    tmp = tmp.replace(",", "")
    views[idx] = int(tmp)

In [21]:
# 모두 같은 길이임을 확인
print(len(video_title))
print(len(video_length))
print(len(views))

4680
4680
4680


In [31]:
import pandas as pd

# 이를 통해 데이터프레임 생성.
df = pd.DataFrame({'title': video_title,'length': video_length ,'views': views})
df.head()

,title,length,views
0,침착맨의 취향,13:43,300590
1,인싸들 먹거리 탐구시간,19:54,528856
2,음식 속 최고의 '힘숨찐' 재료 월드컵,21:44,458111
3,침착맨 스튜디오에 놀러온 승우아빠 님,17:08,348355
4,침투부 100만 특집기획 '침착맨이 듣는다',32:05,456495


In [37]:
# 영상 시간 데이터를 범주형 데이터로 바꾸기 위한 함수 작성

def classifyLength(length):
    tmp = length.split(":")
    if len(tmp) == 1:
        return '5분 미만'
    elif len(tmp) == 2:
        if int(tmp[0]) < 5:
            return '5분 미만'
        elif int(tmp[0]) < 15:
            return '5분 ~ 15분'
        elif int(tmp[0]) < 30:
            return '15분 ~ 30분'
        else:
            return '30분 ~ 1시간'
    elif len(tmp) == 3:
        if int(tmp[0]) < 2:
            return '1시간 ~ 2시간'
        else:
            return '2시간 이상'
    else: # len(tmp) >= 4
        return '2시간 이상'

In [38]:
# 영상 시간 데이터를 범주형 데이터로 변환
df['length'] = df['length'].apply(classifyLength)
df.head()

,title,length,views
0,침착맨의 취향,5분 ~ 15분,300590
1,인싸들 먹거리 탐구시간,15분 ~ 30분,528856
2,음식 속 최고의 '힘숨찐' 재료 월드컵,15분 ~ 30분,458111
3,침착맨 스튜디오에 놀러온 승우아빠 님,15분 ~ 30분,348355
4,침투부 100만 특집기획 '침착맨이 듣는다',30분 ~ 1시간,456495


In [45]:
# 범주화 시킨 데이터를 One-hot encoding 진행

encoded_length =  pd.get_dummies(df['length'])
df = pd.concat([df['title'], encoded_length, df['views']], axis=1)
df.head()

,title,15분 ~ 30분,1시간 ~ 2시간,2시간 이상,30분 ~ 1시간,5분 ~ 15분,5분 미만,views
0,침착맨의 취향,0,0,0,0,1,0,300590
1,인싸들 먹거리 탐구시간,1,0,0,0,0,0,528856
2,음식 속 최고의 '힘숨찐' 재료 월드컵,1,0,0,0,0,0,458111
3,침착맨 스튜디오에 놀러온 승우아빠 님,1,0,0,0,0,0,348355
4,침투부 100만 특집기획 '침착맨이 듣는다',0,0,0,1,0,0,456495


### 03. 영상 제목 형태소 분석, BOW 생성

In [46]:
# konlpy 모듈 사용

import konlpy
from konlpy.tag import Okt
import numpy as np

In [47]:
# 단어를 토큰화, 불용어를 제거할 수 있는 함수 작성.

twitter_tag = Okt()

def twitter_tokenizer(text):
    malist = twitter_tag.pos(text, norm=True, stem=True)
    r = []
    for word in malist:
        if not word[1] in ["Josa", 'Eomi', 'Punctuation', 'KoreanParticle']:
            r.append(word[0])
    return r

한 여기부터 슬슬 잘못되어가고있다는 것을 느끼기 시작했습니다..ㅎ 피드백 부탁드립니다

In [55]:
# train, test 데이터 생성

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = df.drop(['views'], axis=1)
y = df['views']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [56]:
# CounterVectorizer를 이용하여 BOW 생성

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer = twitter_tokenizer).fit(X_train['title'])

C:\Users\timo\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [57]:
# sparse matrix 형태로 생성
tokenized_title = vect.transform(X_train['title'])

<3744x5636 sparse matrix of type '<class 'numpy.int64'>'
	with 40927 stored elements in Compressed Sparse Row format>

In [68]:
# 일반 행렬 형태로 변환 후 X_train데이터 완성

X_train = np.concatenate((tokenized_title.toarray(), X_train.iloc[:, 1:].to_numpy()), axis=1)

In [69]:
# 선형 회귀 모델 생성
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [70]:
#같은 방식으로 test 데이터 생성
tokenized_title_test = vect.transform(X_test['title'])
X_test = np.concatenate((tokenized_title_test.toarray(), X_test.iloc[:, 1:].to_numpy()), axis=1)

In [73]:
from sklearn.metrics import mean_squared_error, r2_score

pred = lr.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)


print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, pred)))

MSE : 4487101517230821128204553593815040.000, RMSE : 66985830719868072.000
Variance score : -56621570492747562876928.000
